*todo:*
- numpy type refactor
- overwriting variable x of fx with x = roulette
- sympy simplify
- numexpr
- deepcopy
- cut in mse same
- type portability of numpy.types
- jit numba

In [12]:
from random import random, choice, randint , choices
from pprint import pprint
from binarytree import Node , tree
import numpy as np
import graphviz
import sympy
import copy
from typing import Dict, List, Set, Tuple
import numexpr as ne
# import logging
from expression_tree import Expression
from mytools import operator_map, expTree2str, evaluate, evaluate_np
from first_population import generate_expressions

In [25]:
debugflag = False

In [14]:
op = {'+', '-', '*', '/', '^',
      'sin', 'cos'}
op_info = {'+': (2, 1), '-': (2, 1),
           '*': (2, 2), '/': (2, 2),
           '^': (2, 3),
           'sin': (1, 4), 'cos': (1, 4)}
assotiation = {'+': 'LR', '-': 'LR',
               '*': 'LR', '/': 'LR',
               '^': 'RL',
               'sin': 'RL', 'cos': 'RL'}
varchar = {'x'}
strscope = 'x'
scope = [c for c in strscope] 
# print(f"{scope=}")
# todo : expand scope


In [15]:
def f(x):
    return x**2 + 2*x

In [16]:
def MSE(actual:np.array, predictions:np.array):
    """Mean Squared Error"""
    # ac, pre        = np.array(actual) ,np.array(predictions)
    ac, pre = actual, predictions
    dif            = np.subtract(ac,pre)
    squared_error  = np.square(dif)
    sum_error      = np.sum(squared_error)
    if debugflag:
        print("''''''''''''''''''''''''''''''''''")
        print(  'ac= '   , ac  ,  '\n',
            'pre= '  , pre ,  '\n',
            'dif= '  , dif ,  '\n',
            'sq= '   ,squared_error,'\n'
            'sum_error= ', sum_error,'\n'
            'len=',len(ac),'\n',
            'final=', sum_error / len(ac)
            )
        print("''''''''''''''''''''''''''''''''''")
    return sum_error / len(ac)
    # sb = [(a1 - b1)**2 for (a1,b1) in zip(actual,predictions)]

In [17]:
def crossover(x: Node, y:Node):
    # print("______________________________________________________")
    # print(x)
    # print(y)
    # print("++++++++++")
    x_copy = copy.deepcopy(x)
    y_copy = copy.deepcopy(y)
    inorder_x = x_copy.inorder
    inorder_y = y_copy.inorder
    # print(inorder_x, '\n',inorder_y)
    x_node = Node(1)
    y_node = Node(1)
    while x_node.value not in op:
        x_node = choice(inorder_x)
    while y_node.value not in op:
        y_node = choice(inorder_y)
    # print(x_node)
    # print(y_node)
    # print('res')
    x_node.__dict__, y_node.__dict__ = y_node.__dict__, x_node.__dict__
    # print(x_copy)
    # print(y_copy)
    return x_copy, y_copy

In [19]:
def evalFx(population:List[Node] ,x:np.array) -> None:
    y_pred = [evaluate_np(root=tree,vars=x) for tree in population]
    if debugflag:
        print("^^^^^^^^^^^^^^^^^^^^^^f(x)")
        for index,tree in enumerate(population):
            print(f"{index}: {str(sympy.sympify(str(expTree2str(tree))))}")
            print(y_pred[index])
    return y_pred


In [18]:
def get_ready_for_selection_np(population, mse):

    sumOfList = np.sum(mse)
    if debugflag:
        print(
            f"^^^^^^^^^^^^^^^^^^^^^^^^^^min:{int(np.min(mse))}  mseSum:{int(sumOfList)} mean:{int(np.mean(mse))}")
    selection_probs = 1 - mse/sumOfList
    return selection_probs


def roulette_wheel_selection_np(population, selection_probs):
    return population[np.random.choice(len(population), p=selection_probs)]


In [27]:
x = np.arange(-5,5)
y = f(x)
print(f"{x=}")
print(f"{y=}")
# todo range from zero but / has problem
# randexpr_arr = generate_expressions(scope, num_exp=5, num_ops=10)
randexpr_arr = ["sin(x)*x","x*x*x*x","x*x*x*x*x","x*x*x*x*x*x*x*x*x","x*x*x*x*x","x*x*x*x*x*x","x*x + 2*x"]
simpled = [str(sympy.sympify(str(x))) for x in randexpr_arr]
# simpled = list({str(sympy.sympify(str(x))) for x in generate_expressions(scope, num_exp=10, num_ops=10)})
for tree in simpled:
    print(tree)
newvals=[]
for exp in simpled:
    newvals.append(ne.evaluate(exp))
newvals

x=array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4])
y=array([15,  8,  3,  0, -1,  0,  3,  8, 15, 24])
x*sin(x)
x**4
x**5
x**9
x**5
x**6
x**2 + 2*x


[array([-4.79462137, -3.02720998,  0.42336002,  1.81859485,  0.84147098,
         0.        ,  0.84147098,  1.81859485,  0.42336002, -3.02720998]),
 array([625, 256,  81,  16,   1,   0,   1,  16,  81, 256], dtype=int32),
 array([-3125, -1024,  -243,   -32,    -1,     0,     1,    32,   243,
         1024], dtype=int32),
 array([-1953125,  -262144,   -19683,     -512,       -1,        0,
               1,      512,    19683,   262144], dtype=int32),
 array([-3125, -1024,  -243,   -32,    -1,     0,     1,    32,   243,
         1024], dtype=int32),
 array([15625,  4096,   729,    64,     1,     0,     1,    64,   729,
         4096], dtype=int32),
 array([15,  8,  3,  0, -1,  0,  3,  8, 15, 24], dtype=int32)]

In [28]:
randexpr_exp = []
randexpr_tree = []
for index, expression in enumerate(randexpr_arr):
    test = Expression(expression=expression, operators=op, operators_info=op_info,
                      operators_associativity=assotiation, variables=varchar)
    randexpr_exp.append(test)
    randexpr_tree.append(test.tree())
    # print(f"{index}:", randexpr_exp[index], "& ", expression)
    # print(randexpr_tree[index])
for tree in randexpr_tree:
    print(str(sympy.sympify(str(expTree2str(tree)))))


sinx*x
x**4
x**5
x**9
x**5
x**6
x**2 + 2*x


In [50]:
def printPopulMse(population,mse=None):
    if mse is None:
        mse = np.zeros(len(population) , dtype=int)
    print("__________________________________________________________________")
    for tree,num in zip(population,mse):
        print("{0:20d}  |  {1}".format(num,str(sympy.sympify(str(expTree2str(tree)))))  )
    print("__________________________________________________________________")
    

In [57]:
def genetic_algo(population: list, mutation_probability: float, saving: float, x: np.array, y: np.array):
    for abc in range(20):
        new_population = []
        y_pred = evalFx(population ,x)
        mse = np.array([int(MSE(y, gg)) for gg in y_pred])
        sortidx = mse.argsort()
        sorted_popul= [population[num] for num in sortidx]
        population = sorted_popul
        mse = np.sort(mse)
        printPopulMse(population,mse)

        new_population.extend(population[:int(saving * len(population))])
        print(f"{len(population)=} || {saving*len(population)=}")
        printPopulMse(new_population)

        len_population = int(len(population) * (1-saving) / 2)
        selection_probs = get_ready_for_selection_np(population, mse)
        for i in range(len_population):
            x_chosen = roulette_wheel_selection_np(population, selection_probs)
            y_chosen = roulette_wheel_selection_np(population, selection_probs)
            (child1, child2) = crossover(x_chosen, y_chosen)
            # if (random() < mutation_probability):
            #     mutate(child)
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
    y_pred = []
    evalFx(population ,x ,y_pred)
    mse = [MSE(y, gg) for gg in y_pred]
    population = [guy for _, guy in sorted(
                    zip(mse, population), reverse=True, key=lambda x: x[0])]
    mse = sorted(mse, reverse=True)
    return population


res = genetic_algo(population=randexpr_tree, saving=0.2,
                   mutation_probability=0.5, x=x, y=y)
print(res[0])
print(expTree2str(res[0]) )
print(str(
            sympy.sympify(
                str(expTree2str(res[0]))
            )
          )
      )
# for tree in res:
#     print(tree)
# simpled = [str(sympy.sympify(str(bruh))) for bruh in res]
# pprint(simpled)


__________________________________________________________________
                   0  |  x**2 + 2*x
                 151  |  sinx*x
               49819  |  x**4
             1203872  |  x**5
             1203872  |  x**5
            27800914  |  x**6
           159139776  |  x**9
__________________________________________________________________
len(population)=7 || saving*len(population)=1.4000000000000001
__________________________________________________________________
                   0  |  x**2 + 2*x
__________________________________________________________________


ValueError: probabilities do not sum to 1

In [ ]:
# # strscope = "abcde"
# # scope = [c for c in strscope]
# # num_exp=randint(1,5),
# # randexpr = generate_expressions(scope,num_exp=int(input("enter num_exp: ")),num_ops=int(input("enter num_ops: ")))
# # for index,expression in enumerate(randexpr):
# #     print(expression)
# #     # print(f'{index=}------>',tree)
# # test = Expression(expression = "x+sin(90)^2*y",
# #                     operators = {'+', 'sin', '^', '*'},
# #                     operators_info = {'+': (2, 1), '*': (2, 2),'^': (2, 3), 'sin': (1, 4)},
# #                     operators_associativity = {'+': 'LR', '*': 'LR','^': 'RL', 'sin': 'RL'},
# #                     variables = {'x', 'y'})
# expr = "x+sin(90)^2*y"
# expr = '1+2*x^9/cos(x^2)+2*y^9'
# expr = '1+2*x^9/cos(x^2)^2+2*y^9'
# expr = "x+2*y^2"
# op = {'+', '-', '*', '/', '^',
#       'sin', 'cos'}
# op_info = {'+': (2, 1), '-': (2, 1),
#            '*': (2, 2), '/': (2, 2),
#            '^': (2, 3),
#            'sin': (1, 4), 'cos': (1, 4)}
# assotiation = {'+': 'LR', '-': 'LR',
#                '*': 'LR', '/': 'LR',
#                '^': 'RL',
#                'sin': 'RL', 'cos': 'RL'}
# varchar = {'x', 'y'}
# test = Expression(expression=expr, operators=op, operators_info=op_info,
#                   operators_associativity=assotiation, variables=varchar)
# print(expr)
# print(test.tree())
# test.tree().graphviz()
# # pprint(test._tokens)
# # pprint(list(expr))
# print(evaluate(test.tree(), {'x': 4, 'y': 2}))

In [ ]:
# test.tree().graphviz()